In [1]:
from configparser import ConfigParser
import os
import pydub
import matplotlib.pyplot as plt
import shutil
from scipy.io import wavfile
from tempfile import mktemp
from opensoundscape.audio import Audio
from opensoundscape.spectrogram import Spectrogram
from IPython.display import clear_output
import random
from main import download_playlist

In [4]:
pl_link = "https://open.spotify.com/playlist/37i9dQZF1DXaXB8fQg7xif?si=9898256e6779497c"

config = ConfigParser()
config.read('config.ini')
playlist_dl_dir = config['MAIN']['playlist_dl_dir']
dl_string = f"spotify_dl -o {playlist_dl_dir}"
os.system(dl_string+f" -l {pl_link}")

0

In [3]:
config = ConfigParser()
config.read('config.ini')
playlist_dl_dir = config['MAIN']['playlist_dl_dir']
jazz_playlist_dir = playlist_dl_dir + "\\jazz"
classical_pl_dir = playlist_dl_dir + "\\classical"
pop_pl_dir = playlist_dl_dir + "\\pop"
rock_pl_dir = playlist_dl_dir + "\\rock"
dance_pl_dir = playlist_dl_dir + "\\dance"
print(jazz_playlist_dir)

D:\uni\deepLearning\musicDataset\jazz


In [4]:
folder = classical_pl_dir

renaming files

In [6]:
def rename_files(folder):
    if not os.path.exists(os.path.join(folder, "renamed")):
        os.mkdir(os.path.join(folder, "renamed"))
    files = [f for f in os.listdir(folder)]
    for count, name in enumerate(files):
        if name.endswith(".txt") or os.path.isdir(os.path.join(folder, name)):
            continue
        new_name = f"audio_{count}.mp3"
        shutil.copy(os.path.join(folder, name), os.path.join(folder, "renamed", new_name))

files shortening

In [13]:
def shorten_audio(folder, audio_len=0.5):
    renamed_folder = os.path.join(folder, "renamed")
    renamed_files = os.listdir(renamed_folder)
    shortened_folder = os.path.join(folder, "shortened")
    if not os.path.exists(shortened_folder):
        os.mkdir(shortened_folder)
    for count, file in enumerate(renamed_files):
        filePath = os.path.join(renamed_folder, file).replace(".mp3", "")
        saveFilepath = os.path.join(shortened_folder, file)
        if file in shortened_folder or not os.path.isfile(filePath):
            continue
        sound = pydub.AudioSegment.from_mp3(filePath)
        for n in range(3):
            extract = sound[0+(0.5*60*1000*n):audio_len*60*1000+(0.5*60*1000*n)]
            extract.export(saveFilepath+"_"+str(n)+".mp3", format="mp3")
        clear_output(wait=True)
        print(f"{count}/{len(renamed_files)}")
    print(f"shortening {folder} done!")

Spectrogram generation

In [8]:
def generate_spectrograms(folder, spectrogram_shape=(200, 500)):
    shortened_folder = os.path.join(folder, "shortened")
    spectrogram_folder = os.path.join(folder, "spectrogram")
    if not os.path.exists(spectrogram_folder):
        os.mkdir(spectrogram_folder)
    files = os.listdir(shortened_folder)
    for count, file in enumerate(files):
        audio_path = os.path.join(shortened_folder, file)
        im_save_path = os.path.join(spectrogram_folder, f"{count+1}.png")
        audio = Audio.from_file(audio_path)
        spectrogram = Spectrogram.from_audio(audio)
        image = spectrogram.to_image(spectrogram_shape)
        image.save(im_save_path)
        clear_output(wait=True)
        print(f"{count+1}/{len(files)}")
    print(f"all spectrograms in {folder} created")


Dataset creation

In [14]:
main_folder = "D:\\uni\\deepLearning\\musicDataset2"
ganre_names = [f for f in os.listdir(main_folder) if not os.path.isfile(os.path.join(main_folder, f))]
train_val_split = (8/10)

for g in ganre_names:
    ganre_folder = os.path.join(main_folder, g)

    # rename_files(ganre_folder)
    shorten_audio(ganre_folder)
    generate_spectrograms(ganre_folder)

    spectrogram_folder = os.path.join(ganre_folder, "spectrogram")
    spectrogram_files = [f for f in os.listdir(spectrogram_folder) if os.path.isfile(os.path.join(spectrogram_folder, f))]

    train_dir = os.path.join(spectrogram_folder, "train")
    test_dir = os.path.join(spectrogram_folder, "test")
    if not os.path.exists(train_dir) or not os.path.exists(test_dir):
        os.makedirs(train_dir)
        os.makedirs(test_dir)
    
    random.shuffle(spectrogram_files)
    train_files = spectrogram_files[0:int(len(spectrogram_files)*train_val_split)]
    for f in train_files:
        shutil.move(os.path.join(spectrogram_folder, f), os.path.join(train_dir, f))
        spectrogram_files.remove(f)
    for f in spectrogram_files:
        shutil.move(os.path.join(spectrogram_folder, f), os.path.join(test_dir, f))
    print(f"Splitting done in {g} folder")
    

30/59


KeyboardInterrupt: 

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Activation, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle

compile_model = True
save_scalars = False
load_older_model = False

learning_rate = 0.0001
first_neurons = 128
second_neurons = 64
third_neurons = 0
fourth_neurons = 0

models_no = f"{first_neurons}_{second_neurons}"
if third_neurons != 0:
    models_no += f"_{third_neurons}"
if fourth_neurons != 0:
    models_no += f"_{fourth_neurons}"
model_path = f"models_{models_no}"

In [13]:
from keras_preprocessing.image import load_img, img_to_array, array_to_img
from keras.preprocessing.image_dataset import image_dataset_from_directory

genre_folder = config['MAIN']['playlist_dl_dir']
genre_list = [g for g in os.listdir(genre_folder) if os.path.isdir(os.path.join(genre_folder, g))]
no_genres = len(genre_list)

dataset_dir = "D:\\uni\\deepLearning\\kerasDataset"
class_names = {"classical", "pop", "rock"}

dataset = image_dataset_from_directory(dataset_dir, class_names=class_names, color_mode="grayscale")

for count, g in enumerate(genre_list):
    train_dir = os.path.join(genre_folder, g, 'spectrogram', 'train')
    test_dir = os.path.join(genre_folder, g, 'spectrogram', 'test')
    train_spectrogram_paths = [os.path.join(train_dir, s) for s in os.listdir(train_dir) if s.endswith('.png')]
    test_spectrogram_paths = [os.path.join(test_dir, s) for s in os.listdir(test_dir) if s.endswith('.png')]
    for f in train_spectrogram_paths:
        img = load_img(f)
        img_array = img_to_array(img)
        print(img_array.shape)
        break
    

(200, 500, 3)
(200, 500, 3)
(200, 500, 3)
